In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import dill
import numpy as np
import scipy.io as io
import datetime

sys.path.append('../helper_functions')
sys.path.append('helper_functions')
sys.path.append('../objects')
sys.path.append('objects')

In [3]:
from FishGeneration import FishGeneration
from Fish_IceCreamCone import IceCreamConeFish
from Fish_Capsule import CapsuleFish

from assertion_tests import run_tests
run_tests()

'Success!'

## Load fish details

#### the EOD waveform

In [ ]:
# use a dummy EOD until real EOD is available in this framework
full_eod = np.linspace(0, 2 * np.pi, 1000)
full_eod = np.sin(full_eod) * full_eod * np.exp(-((full_eod - np.pi) ** 2) / (2 * (np.pi / 6) ** 2))

#### the receptor filters

In [ ]:
# use dummy filters until real filters are available in this framework
receptor_filters = np.random.randn(2, *full_eod.shape)
receptor_filters.shape

#### check the input arguments for the wanted fish object (helps with creating fish objects)

In [ ]:
for x in CapsuleFish._initialize_input_argument_names():
    print(f'{x},')

## Dataset: YYYY_MM_DD Purpose

## Dataset: 2024_05_23 Test

In [ ]:
print(datetime.datetime.now().strftime('%Y_%m_%d'))

In [ ]:
dataset_name = 'data-2024_05_23-test'

#### create the Fish object

In [ ]:
base_fish = CapsuleFish(
    receptor_filters=receptor_filters,
    eod_wave_form=full_eod,
    skin_resistivity=(3, "k", "c.2"),  # 3 kOhm.cm^2
    sampling_rate=(2.5, "M"),  # 2.5 MHz,
    nose_position=[0, 0, 0],
    fish_length=(15, "c"),
    angle_yaw=(0, "deg"),
    angle_pitch=(0, "deg"),
    angle_roll=(0, "deg"),
    relative_bend_locations_percentage=np.array([]),
    relative_bend_angle_lateral=(np.array([]), "deg"),
    relative_bend_angle_dorso_ventral=(np.array([]), "deg"),
    # receptors_init=dict(
    #     method="grid_uniDense",
    #     head_t=0,
    #     head_u=0,
    #     tail_t=0,
    #     tail_u=0,
    #     body_t=60,
    #     body_u=30,
    # ),
    # vertical_semi_axis=(2.0, "c"),
    # lateral_semi_axis=(0.9, "c"),
    # rostrocaudal_semi_axis=0,
    # rostrocaudal_semi_axis_tail=0,
    receptors_init=dict(
        method="manual",
        locations=np.array([[15e-2 - 18e-3, 9e-3, 0], [15e-2 - 20e-3, 9e-3, 0], [15e-2 - 22e-3, 9e-3, 0]]).reshape(
            -1, 3
        ),
        normals=np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]]).reshape(-1, 3),
    ),
)

dill.dump(base_fish, open(f"base-fish/{dataset_name}.pkl", "wb"), protocol=4)
base_fish = dill.load(open(f"base-fish/{dataset_name}.pkl", "rb"))

#### visualize fish object for inspection

In [ ]:
base_fish.visualize_scatter(show_normals=2, show_point_currents=3, fig_width=1000, fig_height=350)
print(base_fish.get_N_receptors())

#### select simulation parameters

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict["save_name"]                       = dataset_name
# aquarium properties
data_params_dict["water_conductivities"]            = [0.01]
data_params_dict["boundary_displacements"]          = [[-1e9], [-30e-3]]
data_params_dict["boundary_normals"]                = [[0, 0, 1], [0, 0, 1]]
# fish properties
data_params_dict["fish_obj"]                        = base_fish
data_params_dict["fish_bend_location_percentages"]  = [[0]]
data_params_dict["fish_bend_angle_lateral"]         = [[0], [20]]
data_params_dict["fish_bend_angle_dorso_ventral"]   = [[0]]
data_params_dict["fish_yaw"]                        = [0]
data_params_dict["fish_pitch"]                      = [0]
data_params_dict["fish_roll"]                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 10)) * 1e3
data_params_dict["worm_capacitances"]               = np.exp(np.linspace(np.log(2), np.log(100), 10)) * 1e-10
data_params_dict["worm_radii"]                      = np.array([5, 12.5]) * 1e-3
data_params_dict["worm_position_xs"]                = np.array([-30]) * 1e-3
data_params_dict["worm_position_ys"]                = np.array([5, 10]) * 1e-3 + base_fish.get_lateral_semi_axis() + data_params_dict["worm_radii"]
data_params_dict["worm_position_zs"]                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open(f"params/{data_params_dict['save_name']}.pkl", "wb"), protocol=4)

In [49]:
import pandas as pd

In [83]:

a = pd.DataFrame()
a = pd.concat([a, pd.DataFrame(dict(cond=[1,4,3,6], test=[0,2,5,3]))], axis=0, ignore_index=True)
a = pd.concat([a, pd.DataFrame(dict(cond=[11,12], test=list(np.random.randn(2,3))), index=[17,19])], axis=0, ignore_index=True)
# a.index=np.arange(4)+11

In [84]:
for i, r in a.iterrows():
    print(i)
    print(r)
    print(r['cond'])
    break

0
cond    1
test    0
Name: 0, dtype: object
1


In [90]:
a["test"][4], a

(array([-0.14019952, -0.61735441, -1.01532722]),
    cond                                               test
 0     1                                                  0
 1     4                                                  2
 2     3                                                  5
 3     6                                                  3
 4    11  [-0.14019951824796612, -0.6173544103108182, -1...
 5    12  [0.19762265713741098, -0.21229380681351392, -2...)

In [76]:
a.shape, len(a)

((5, 2), 5)

In [77]:
for i, x in enumerate(a["test"]):
    print(i, x)

0 0
1 2
2 5
3 3
4 None


In [78]:
for i, x in a.iterrows():
    print(i)
    print(x)
    print()

0
cond    1
test    0
Name: 0, dtype: object

1
cond    4
test    2
Name: 1, dtype: object

2
cond    3
test    5
Name: 2, dtype: object

3
cond    6
test    3
Name: 3, dtype: object

4
cond      11
test    None
Name: 4, dtype: object



In [79]:
import torch
temp1 = np.random.randn(10, 3)
temp2 = torch.Tensor(temp1)
print(temp1.dtype, temp2.dtype)

float64 torch.float32


In [80]:
a

,cond,test
0,1,0
1,4,2
2,3,5
3,6,3
4,11,None


In [95]:
time.gmtime()

time.struct_time(tm_year=2024, tm_mon=5, tm_mday=24, tm_hour=19, tm_min=26, tm_sec=15, tm_wday=4, tm_yday=145, tm_isdst=0)

In [96]:
time.now()

AttributeError: module 'time' has no attribute 'now'

In [108]:
(datetime.datetime.now() + datetime.timedelta(seconds=1000)).strftime('%Y_%m_%d-T-%H:%M:%S')

'2024_05_24-T-15:46:06'

In [111]:
3734 // 3600, 3734 % 3600 // 60, 3734.23 % 60

(1, 2, 14.230000000000018)

In [114]:
[list(x) for x in np.linspace(-80,80,17).reshape(-1,1)] + [[15,5]] + [[]]

[[-80.0],
 [-70.0],
 [-60.0],
 [-50.0],
 [-40.0],
 [-30.0],
 [-20.0],
 [-10.0],
 [0.0],
 [10.0],
 [20.0],
 [30.0],
 [40.0],
 [50.0],
 [60.0],
 [70.0],
 [80.0],
 [15, 5],
 []]

In [112]:
from pathlib import Path
Path("processed/dset1").mkdir(parents=True, exist_ok=True)

In [113]:
Path("processed/dset1").mkdir(parents=True, exist_ok=True)

In [42]:
import h5py
import time

d1 = 10000
d2 = 100
d3 = 3

t0= time.time()
my_arr = np.random.randn(d1, d2, d3).astype(np.float64)
t1= time.time()
print(f"Time to generate: {t1 - t0}")

t0 = time.time()
with h5py.File(f"/Users/denisturcu/Desktop/test.hdf5", "w") as f:
    dset = f.create_dataset(
        "tst",
        (d1, d2, d3),
        dtype=np.float32,
    )
    dset[:] = my_arr
t1 = time.time()
print(f"Time to write: {t1 - t0}")

t0 = time.time()
for i in range(d1):
    with h5py.File(f"/Users/denisturcu/Desktop//individual/test-{i}.hdf5", "w") as f:
        dset = f.create_dataset(
            "tst",
            (d2, d3),
            dtype=np.float32,
        )
        dset[:] = my_arr[i]
t1 = time.time()
print(f"Time to write individual: {t1 - t0}")
del my_arr

Time to generate: 0.03670191764831543
Time to write: 0.007992029190063477
Time to write individual: 3.2821009159088135


In [43]:
f = h5py.File(f"/Users/denisturcu/Desktop//test.hdf5", "r")
temp_ids = np.sort(np.random.permutation(d1)[:5000])
t0 = time.time()
temp = f["tst"][temp_ids]
t1 = time.time()
print(f"Time to read: {t1 - t0}")

t0 = time.time()
temp2 = np.zeros((len(temp_ids), d2, d3), dtype=np.float32)
for i, ii in enumerate(temp_ids):
    temp2[i] = f["tst"][ii]
t1 = time.time()
print(f"Time to read 2: {t1 - t0}")

t0 = time.time()
temp3 = np.zeros((len(temp_ids), d2, d3), dtype=np.float32)
for i, ii in enumerate(temp_ids):
    f = h5py.File(f"/Users/denisturcu/Desktop//individual/test-{ii}.hdf5", "r")
    temp3[i] = f["tst"][:]
t1 = time.time()
print(f"Time to read individual: {t1 - t0}")
f.close()

Time to read: 0.2669639587402344
Time to read 2: 0.39317989349365234
Time to read individual: 0.7796530723571777


In [39]:
# f = h5py.File(f"/Users/denisturcu/Desktop//test.hdf5", "r")
# t0 = time.time()
# temp = f["tst"][d1-40000:]
# t1 = time.time()
# print(f"Time to read: {t1 - t0}")

Time to read: 0.0018780231475830078


In [47]:
f.close()

In [44]:
f["tst"][:]

ValueError: Invalid location identifier (invalid location identifier)

In [48]:
type(np.float32)

type

In [13]:
# Time to read: 0.03277420997619629
# Time to read 2: 0.08377718925476074
# Time to read individual: 0.16527891159057617

In [117]:
np.diff([50], prepend=0, append=100) * 15 / 100

array([7.5, 7.5])

In [118]:
len([])

0

## Misc - previous param dicts for reference

In [ ]:
# Create Data Params Dict
data_params_dict = {}
# data_params_dict['save_file_name']                  = 'data_240411_SpatialProcessing_MovingObject_7by15'
data_params_dict['save_file_name']                  = 'data_240425_Detection_1050_receptors_HighContrast'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
# data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e3
data_params_dict['worm_resistances']                = np.array([2e3, 100e3])  # HighContrast
data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e-10  # HighContrast when paired with HighContrast resistances
# data_params_dict['worm_capacitances']               = np.array([2e-10])  # HighContrast
data_params_dict['worm_radii']                      = np.array([12.5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3   # - np.arange(101) * 3e-4
data_params_dict['worm_position_ys']                = (5 * np.power(np.sqrt(1.33), np.arange(25))).round(0) * 1e-3 + base_fish.lat_ax + data_params_dict['worm_radii']
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_231116_SpatialProcessing_MovingObject_11by25'
# aquarium properties
data_params_dict['water_conductivities']            = [0.01]  # np.array([100, 300, 600]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
                                                      # [[-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3),
                                                      #  [-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3), 
                                                      #  [-1e9]+list(-np.arange(35, 70, 5)[::-1]*1e-3)]
                                                    #  [[-1e9]+list(-np.linspace(5*base_fish.roc_ax, base_fish.roc_ax, 9)), ]  # 
                                                    #    [-1e9]+list(-np.linspace(0.058, 0.022, 9)),
                                                    #    [-1e9]+list(-np.linspace(0.058, 0.022, 9))]
data_params_dict['boundary_normals']                =  [[0,0,1]]
                                                      # [[0,0,1], [0,-1,0], [0,1,0]]
      #  [[0,0,1], ]  #
      #    [0,-1,0],
      #    [0,1,0]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]#[[x] for x in np.linspace(-60,60,25)]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]#[[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]#[[60]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e3
data_params_dict['worm_capacitances']               = np.exp(np.linspace(np.log(2), np.log(100), 5)) * 1e-10
data_params_dict['worm_radii']                      = np.array([12.5]) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3 - np.arange(11) * 3e-3
data_params_dict['worm_position_ys']                = np.array([5,10,15,30,50,70,100,150]) * 1e-3 + base_fish.lat_ax + data_params_dict['worm_radii']
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)

In [ ]:
# Create Data Params Dict
data_params_dict = {}
data_params_dict['save_file_name']                  = 'data_240311_falloff_with_distance_and_radius'
# aquarium properties
data_params_dict['water_conductivities']            = np.array([100]) * 1e-6 / 1e-2
data_params_dict['boundary_displacements']          = [[-1e9]]
data_params_dict['boundary_normals']                =  [[0,0,1]]
# fish properties
data_params_dict['fish_obj']                        = base_fish
data_params_dict['fish_bend_angle_lateral']         = [[0]]
data_params_dict['fish_bend_angle_dorso_ventral']   = [[0]]
data_params_dict['fish_bend_location_percentages']  = [[0]]
data_params_dict['fish_yaw']                        = [0]
data_params_dict['fish_pitch']                      = [0]
data_params_dict['fish_roll']                       = [0]
# worm properties
data_params_dict['worm_resistances']                = np.array([500,50000])
data_params_dict['worm_capacitances']               = np.array([2e-10])
data_params_dict['worm_radii']                      = np.arange(5,11) * 1e-3
data_params_dict['worm_position_xs']                = np.array([-30]) * 1e-3
data_params_dict['worm_position_ys']                = np.arange(10,201) * 1e-3 + base_fish.lat_ax
data_params_dict['worm_position_zs']                = np.array([0]) * 1e-3
# save params file
dill.dump(data_params_dict, open('../data/params/' + data_params_dict['save_file_name'] + '_params_dict.pkl', 'wb'), protocol=4)